In [89]:
#import pandas as pd 
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))



[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [90]:
import numpy as np 

In [91]:
print("Hello ese")

Hello ese


In [92]:
!python --version
print('Numpy ' + np.__version__)
print('TensorFlow ' + tf.__version__)

Python 3.12.3
Numpy 2.3.3
TensorFlow 2.20.0


In [93]:

import torch 
import torch.nn as nn 
print(torch.__version__)

2.8.0+cu128


In [94]:


class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, stride, bias=False, padding_mode="reflect"), 
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2), 
        )

    def forward(self, x):
        return self.conv(x)
    
class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]): 
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels*2, features[0], kernel_size=4, stride=2, padding=1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),  
        )

        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(
                CNNBlock(in_channels, feature, stride=1 if feature == features[-1] else 2),
            ) 
            in_channels = feature

        
        layers.append(
            nn.Conv2d(
                in_channels, 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect"
            ),
        )


        self.model = nn.Sequential(*layers)

    def forward(self, x, y): 
        x = torch.cat([x, y], dim=1)
        x = self.initial(x)
        return self.model(x)


In [95]:
def test():
    x = torch.randn((1, 3, 256, 256))
    y = torch.randn((1, 3, 256, 256))
    model = Discriminator()
    preds = model(x, y)
    print(preds.shape)



In [96]:
if __name__ == "__main__":
    test()

torch.Size([1, 1, 26, 26])


In [97]:
###################################

In [ ]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, act="relu", use_dropout=False):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False, padding_mode="reflect")
            if down 
            else nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias = False), 
            nn.BatchNorm2d(out_channels),
            nn.ReLU() if act == "relu" else nn.LeakyReLU(0.2), 

        )
        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.conv(x)
        return self.dropout(x) if self.use_dropout else x 
    

class Generator(nn.Module):
    def __init__(self, in_channels = 3, features=64): 
        super().__init__()   
        self.initial_down = nn.Sequential(
            nn.Conv2d(in_channels, features, 4, 2, 1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),

        ) #128 after this one 

        self.down1 = Block(features, features*2, down=True, act="leaky", use_dropout=False )    #64x64 
        self.down2 = Block(features*2, features*4, down=True, act="leaky", use_dropout=False )  #32x32 
        self.down3 = Block(features*4, features*8, down=True, act="leaky", use_dropout=False )  #16
        self.down4 = Block(features*8, features*8, down=True, act="leaky", use_dropout=False )  #8
        self.down5 = Block(features*8, features*8, down=True, act="leaky", use_dropout=False )  #4 
        self.down6 = Block(features*8, features*8, down=True, act="leaky", use_dropout=False )  #2
        self.bottleneck = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1, padding_mode="reflect"), nn.ReLU(), #1x1
        )
        self.up1 = Block(features*8, features*8, down=False, act="relu", use_dropout=True) #2x2  
        self.up2 = Block(features*8*2, features*8, down=False, act="relu", use_dropout=True) #4x4
        self.up3 = Block(features*8*2, features*8, down=False, act="relu", use_dropout=True) 
        self.up4 = Block(features*8*2, features*8, down=False, act="relu", use_dropout=False)
        self.up5 = Block(features*8*2, features*4, down=False, act="relu", use_dropout=False)
        self.up6 = Block(features*4*2, features*2, down=False, act="relu", use_dropout=False)
        self.up7 = Block(features*2*2, features*2, down=False, act="relu", use_dropout=False)
        self.final_up = nn.Sequential(
            nn.ConvTranspose2d(features*2, in_channels, kernel_size=4, stride=2, padding=1), 
            nn.Tanh(), 
        )

    def forward(self, x):
        d1 = self.initial_down(x)
        d2 = self.down1(d1)
        d3 = self.down1(d2)
        d4 = self.down1(d3)
        d5 = self.down1(d4)
        d6 = self.down1(d5)
        d7 = self.down1(d6)
        bottleneck = self.bottleneck(d7)
        up1 = self.up1(bottleneck)
        up2 = self.up2(torch.cat([up1, d7], 1))
        up3 = self.up3(torch.cat([up2, d6], 1))
        up4 = self.up4(torch.cat([up3, d5], 1))
        up5 = self.up5(torch.cat([up4, d4], 1))
        up6 = self.up6(torch.cat([up5, d3], 1))
        up7 = self.up7(torch.cat([up6, d2], 1))
        return self.final_up(torch.cat([up7, d1 ], 1))
        




      


In [99]:
def test1(): 
    x = torch.randn((1, 3, 256, 256))
    model = Generator(in_channels=3, features=64)
    preds= model(x)
    print(preds.shpae)




In [100]:
if __name__ == "__main__":
    test1()

RuntimeError: Given groups=1, weight of size [128, 64, 4, 4], expected input[1, 128, 66, 66] to have 64 channels, but got 128 channels instead